<a href="https://colab.research.google.com/github/nishantparmar24/Political-Risk/blob/master/NewsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install newsapi-python

  Created wheel for newsapi-python: filename=newsapi_python-0.2.5-py2.py3-none-any.whl size=6593 sha256=ec4539e425076d4a7170384dece99755c4e9bcd0c4683159c06d0184a335ce2d
  Stored in directory: /root/.cache/pip/wheels/d8/dc/5e/857ef20410a023cfe728ac6f360958df44b199b459cbc6ccbc
Successfully built newsapi-python


In [0]:
from datetime import datetime as dt
from google.colab import drive
from newsapi import NewsApiClient
from urllib.parse import quote
import json
import os
import pandas as pd
import re
import requests

In [0]:
drive.mount('/gdrive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/


In [0]:
output_directory = '/gdrive/My Drive/Political Risk Project/Test Data/News Results/'
API_usage_file = os.path.join(output_directory, 'api_usage.json')

In [0]:
def get_api_usage(date):
    if os.path.lexists(API_usage_file):
        api_usage = pd.read_json(API_usage_file, orient='records')
        if date in api_usage.columns:
            return api_usage, api_usage[date].squeeze()
        else:
            update_api_count(date)
            return api_usage, 1
    return None, None

In [0]:
def update_api_count(date):
    api_usage, count = get_api_usage(date)
    if not api_usage.empty:
        updated_count = 0
        if date in api_usage.columns:
            api_usage[date] += 1
            updated_count = api_usage[date].squeeze()
        else:
            api_usage = 0
        api_usage.to_json(API_usage_file, orient='records', date_format='iso')
        sleep(3)
        _, new_count = get_api_usage(date)
        print("Updating new API usage...")
        # print(new_count)
        if new_count != updated_count:
            print("...")
            drive.mount("/gdrive", force_remount=True)
        _, new_count = get_api_usage(date)
        print("API usage count updated! Current usage: {}".format(new_count))


In [0]:
format_date_str = lambda s: "0{}".format(s) if int(s) < 10 else "{}".format(s)
today = lambda: "{}-{}-{}".format(dt.now().year, 
                                  format_date_str(dt.now().month), 
                                  format_date_str(dt.now().day))
today_hm = lambda: "{}_{}-{}".format(today(),
                                     format_date_str(dt.now().hour),
                                     format_date_str(dt.now().minute))

In [0]:
r, c = get_api_usage(today())
r

,2019-09-30T00:00:00.000Z,2019-10-01T00:00:00.000Z,2019-10-18,2019-10-24
0,6,10,1,12


In [0]:
def save_article(article_text, filename):
    out_file = os.path.join(output_directory, filename)
    with open(out_file, "w") as output:
        output.write(article_text)
    if os.path.lexists(out_file):
        return out_file
    return None

In [0]:
def get_api_key():
    src_ = "/gdrive/My Drive/Political Risk Project/NewsAPIKey/"
    with open(os.path.join(src_, "news_api_key.json")) as key:
        obj = json.load(key)
        return obj["api_key"]

In [0]:
newsapi = NewsApiClient(api_key=get_api_key())

In [0]:
sources = ",".join(["reuters", "ansa", "google-news-it"])
print(sources)
queries = ["early election", "snap election", "government collapse", 
           "government coalition", "election", "instability", "uncertainty",
           "crisis", "coalition"]
country = "it"
page_size = 100
endpoint = "everything" # "top-headlines"

reuters,ansa,google-news-it


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


API usage count updated! Current usage: 10


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


API usage count updated! Current usage: 11
API usage count updated! Current usage: 12


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [0]:
def fetch_news(queries, sources, page_size=100, **kwargs):
    top_headlines_map = dict()
    # saved_files = list()
    news_collection = pd.DataFrame()
    for query in queries:
        headlines = newsapi.get_everything(q=query,
                                           sources=sources,
                                           sort_by='relevancy',
                                           page_size=page_size)
        if headlines["totalResults"] > 0:
            # response_text = json.loads(headlines.text)
            all_articles = headlines["articles"]
            collection = list()
            for article in all_articles:
                data_ = dict()
                source = article.pop("source")
                content = article.pop("content")
                source = {"source": source["id"]}
                article.update(source)
                collection.append(article)
            data_df = pd.DataFrame(data=collection)
            if not data_df.empty:
                match_str = '({})'.format(query)
                reqd_df = data_df[data_df['description'].str.contains(
                    match_str)]
                if news_collection.empty:
                    news_collection = reqd_df
                else:
                    news_collection = news_collection.append(reqd_df)
                # responses_file = "{}_on_{}.csv".format(endpoint,
                #                                       query.replace(" ", "_"))
                # top_headlines_map[query] = reqd_df
            
                # out_file = os.path.join(output_directory, responses_file)
                # data_df.to_csv(out_file, index=False)
            update_api_count(today())
    return news_collection


In [0]:
news_collection = fetch_news(queries, sources)
# news_collection.to_csv(os.path.join(output_directory, 'all_results_new.csv'), 
#                        index=False)

In [0]:
news_collection = news_collection.reset_index().drop(columns=['index'])

In [0]:
enpoint_uri = "http://ac6a2064dee3c11e99ced0a13821e56d-733867741.ap-southeast-1.elb.amazonaws.com/sentiment/classifier"
headers = {"content-type": "application/json"}

In [0]:
def get_polarity(sentence):
    data = json.dumps({"polarity": sentence})
    response = requests.post(enpoint_uri, data=data, headers=headers)
    return response.json() if response.status_code == 200 else ""

In [0]:
news_collection["polarity_v1"] = news_collection["title"].apply(get_polarity)
news_collection

,author,description,publishedAt,source,title,url,urlToImage,polarity_v1
0,Reuters Editorial,Britain's Labour Party will only back an early...,2019-10-22T14:00:52Z,reuters,UK's Labour wants Brexit extension before it w...,https://www.reuters.com/article/us-britain-eu-...,https://s3.reutersmedia.net/resources/r/?m=02&...,negative
1,Marco Aquino,Peruvian opposition lawmakers shelved a snap e...,2019-09-26T20:34:16Z,reuters,Peruvian Congress shelves Vizcarra's proposal ...,https://www.reuters.com/article/us-peru-politi...,https://s3.reutersmedia.net/resources/r/?m=02&...,negative
2,Padraic Halpin,Supporters of Irish Prime Minister Leo Varadka...,2019-10-18T15:50:33Z,reuters,Irish PM's party primed for snap election if B...,https://www.reuters.com/article/us-ireland-pol...,https://s4.reutersmedia.net/resources/r/?m=02&...,positive
3,Reuters Editorial,Iraqi populist cleric Moqtada al-Sadr calls on...,2019-10-04T19:38:48Z,reuters,Iraqi populist cleric Moqtada al-Sadr calls fo...,https://www.reuters.com/article/us-iraq-protes...,https://s4.reutersmedia.net/resources_v2/image...,negative
4,Fatos Bytyci,Voters in Kosovo will pick a new parliament on...,2019-10-01T16:00:54Z,reuters,Kosovo to elect new parliament amid anger over...,https://www.reuters.com/article/us-kosovo-elec...,https://s4.reutersmedia.net/resources_v2/image...,negative


In [0]:
def save_output(news_collection, 
                curr_datetime):
    file_name = 'newsheadlines_predictions_{}.csv'.format(curr_datetime)
    news_collection.to_csv(os.path.join(output_directory, file_name),
                           index=False)

In [0]:
save_output(news_collection, today_hm())

In [0]:
# def update_api_count(date, count_data, count):
#     if not count_data.empty:
#         if date in count_data.columns:
#             count_data[date] = count
#             count_data.to_json(API_usage_file, 
#                                orient='records',
#                                date_format='iso')
#             r, c = get_api_usage(date)
#             while c != count:
#                 print("Updating count...")
#             print("Count updated!")
#         else:
#             print("Date not in usage file!\nCreating an entry")
#             count_data[date] = 1
#             count_data.to_json(API_usage_file, 
#                                orient='records', 
#                                date_format='iso')